## Crowding4 example 

### This notebook serves as a representation of the `bigraph_process.Process` API implementation: `SmoldynProcess`.

#### This example uses the Smoldyn model: `crowding4.txt` found in this repo at `../examples/model_files/crowding_model.txt` to test/validate the `SmoldynProcess` object.

In [1]:
import sys
import os

# Get the current file's directory (demos)
current_dir = os.path.dirname(os.path.realpath('__file__'))

# Go up one level to the root of the repo and then into the smoldyn_process directory
repo_root = os.path.join(current_dir, '..')
smoldyn_process_dir = os.path.join(repo_root, 'smoldyn_process')

# Add the smoldyn_process directory to sys.path
sys.path.append(smoldyn_process_dir)

In [2]:
from typing import *
import numpy as np
import smoldyn as sm
from process_bigraph import Process, Composite, process_registry, types
from smoldyn_process.sed2 import pf

In [3]:
smoldyn_process_dir

'/Users/alex/Desktop/uchc_work/repos/smoldyn-process/smoldyn_process/demos/../smoldyn_process'

In [ ]:
class SmoldynProcess(Process):
    """Smoldyn-based implementation of bi-graph process' `Process` API. Please note the following:
    
    For the purpose of this `Process` implementation, 
        
    at each `update`, we need the function to do the following for each molecule/species in the simulation:
        
        - Get the molecule count with Smoldyn lang: (`molcount {molecule_name}`)
        - Get the molecule positions and relative corresponding time steps, indexed by the molecule name with Smoldyn lang: (`listmols`)[molecule_name]
        - ?Get the molecule state?
        - Kill the molecule with smoldyn lang: (`killmol {molecule_name}`)
        - Add the molecule back to the solution(cytoplasm), effectively resetting it at boundary coordinates with Python API: (`simulation.addMolecules()
        
    PLEASE NOTE:
    
        The current implementation of this class assumes that output commands are not listed in the Smoldyn 
            model file. If they are, simply comment them out before using this.
    
    """

    config_schema = {
        'model_filepath': 'string',
        'animate': 'boolean',
    }

    def __init__(self, config: Dict = None):
        super().__init__(config)

        # specify the model fp for clarity
        self.model_filepath = self.config.get('model_filepath')

        # enforce model filepath passing
        if not self.model_filepath:
            raise ValueError('The config requires a path to a Smoldyn model file.')

        # initialize the simulator from a Smoldyn model.txt file.
        self.simulation: sm.Simulation = sm.Simulation.fromFile(self.model_filepath)

        # add the relevant output files and commands required for the update
        self.simulation.addOutputData('time')
        self.simulation.addCommand(cmd='executiontime time', cmd_type='E')
        self.simulation.addOutputData('molecule_counts')
        self.simulation.addCommand(cmd='molcount molecule_counts', cmd_type='E')
        self.simulation.addOutputData('molecule_locations')
        self.simulation.addCommand(cmd='listmols molecule_locations', cmd_type='E')

        # TODO: Add a handler that checks if self.config.get('molecules') is None, and sets thru Python if not

        # count the num species
        species_count = self.simulation.count()['species']

        # create a list of species objects
        self.species_names: List[str] = []
        for index in range(species_count):
            species_name = self.simulation.getSpeciesName(index)
            self.species_names.append(species_name)

        # get the simulation boundaries, which in the case of Smoldyn denote the physical boundaries
        # TODO: add a verification method to ensure that the boundaries do not change on the next step...
            # ...to be removed when expandable compartment size is possible:
        self.boundaries = self.simulation.getBoundaries()

        # set graphics (defaults to False)
        if self.config['animate']:
            self.simulation.addGraphics('opengl_better')

    def initial_state(self) -> Dict[str, Dict]:
        """Set the initial parameter state of the simulation. NOTE: Due to the nature of this model,
            Smoldyn assigns a random uniform distribution of integers as the initial coordinate (x, y, z)
            values for the simulation. As such, the `set_uniform` method will uniformly distribute
            the molecules according to a `highpos`[x,y] and `lowpos`[x,y] where high and low pos are
            the higher and lower bounds of the molecule spatial distribution.

            NOTE: This method should provide an implementation of the structure denoted in `self.schema`.
        """

        # TODO: update for distribution!
        '''for spec in self.species:
            molecule = self.config['molecules'].get(spec)
            self.simulation.addMolecules(
                species=spec,
                number=molecule['count'],
                pos=molecule['coordinates']
            )'''

        species_dict = {}
        for name in self.species_names:
            species_dict[name] = {
                'time': 0,
                'count': 0,
                'coordinates': None,
            }


        # TODO: fill these with a default state with get initial mol state method
        state = {
            'molecules': species_dict
        }
        return state

    def set_uniform(self, name: str, config: Dict[str, Any]) -> None:
        """Add a distribution of molecules to the solution in
            the simulation memory given a higher and lower bound x,y coordinate. Smoldyn assumes
            a global boundary versus individual species boundaries.
            TODO: If pymunk expands the species compartment, account for
            expanding `highpos` and `lowpos`.

            Args:
                name:`str`: name of the given molecule.
                config:`Dict`: molecule state.
        """
        # get the boundaries
        low_bounds = self.boundaries[0]
        high_bounds = self.boundaries[1]

        # kill the mol, effectively resetting it
        self.simulation.runCommand(f'killmol {name}')

        # redistribute the molecule according to the bounds
        self.simulation.addSolutionMolecules(
            name,
            config['molecules'].get(name)['count'],
            highpos=high_bounds,
            lowpos=low_bounds
        )

    def schema(self) -> Dict:
        """Return a dictionary of molecule names and the expected input/output schema at simulation
            runtime. NOTE: Smoldyn assumes a global high and low bounds and thus high and low
            are specified alongside molecules.
        """
        tuple_type = {'_type': 'tuple', '_apply': 'set'}
        list_type = {'_type': 'list', '_apply': 'set'}
        return {
            'molecules': {
                mol_name: {
                    'time': 'int',
                    'count': 'int', # derived from the molcount output command
                    'coordinates': list_type,
                    #'velocity': tuple_type,  # QUESTION: could the expected shape be: ((0,0), (1,4)) where: ((xStart, xStop), (yStart, yStop)) ie directional?
                    #'mol_type': 'string',
                    #'state': 'string'
                } for mol_name in self.species_names
            }
        }

    def update(self, state: Dict, interval: int) -> Dict:
        """Callback method to be evoked at each Process interval.

            Args:
                state:`Dict`: current state of the Smoldyn simulation, expressed as a `Dict` whose
                    schema matches that which is returned by the `self.schema()` API method.
                interval:`int`: timestep interval at which to provide the update as the output
                    of this method. NOTE: This update is iteratively called with the `Process` API.

            Returns:
                `Dict`: New state according to the update at interval
        """

        # get the molecule configs
        molecules = state['molecules']

        # distribute the mols according to self.boundaries
        for mol_name, mol_state in molecules.items():
            self.set_uniform(mol_name, mol_state)

        # run the simulation for a given interval
        self.simulation.run(
            stop=interval,
            dt=self.simulation.dt
        )
        
        # get the time data, clear the buffer
        time_data = np.array(self.simulation.getOutputData('time', True)).T.tolist()
        
        # get the data, clear the buffer
        counts_data = np.array(self.simulation.getOutputData('molecule_counts', True)).T.tolist()
        
        # get the data based on the commands added in the constructor, clear the buffer
        location_data = np.array(self.simulation.getOutputData('molecule_locations', True)).T.tolist()

        # get the final counts for the update
        final_time = time_data[-1]
        final_counts = counts_data[-1]
        final_locations = location_data[-1]
        molecules = {}
        for index, name in enumerate(self.species_names, 1):
            molecules[name] = {
                'time': final_time,
                'count': int(final_counts[index]) - state['molecules'][name],
                'coordinates': final_locations
            }
        
        # uniformly reset the solution molecules based on the updated count for each molecule
        for species_name in self.species_names:
            self.set_uniform(species_name, molecules[species_name])
        
        return {'molecules': molecules}


# register the process above as the name passed in the first argument below
process_registry.register('smoldyn_process', SmoldynProcess)


def test_process():
    """Test the smoldyn process using the crowding model."""

    # this is the instance for the composite process to run
    instance = {
        'smoldyn': {
            '_type': 'process',
            'address': 'local:smoldyn_process',
            'config': {
                'model_filepath': 'smoldyn_process/examples/model_files/crowding_model.txt',
                'animate': False,
            },
            'wires': {
                'molecules': ['molecules_store'],
            }
        },
        'emitter': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'ports': {
                    'inputs': {
                        'molecules': 'dict'
                    }
                }
            },
            'wires': {
                'inputs': {
                    'molecules': ['molecules_store'],
                }
            }
        }
    }

    # make the composite
    workflow = Composite({
        'state': instance
    })

    # run
    workflow.run(10)

    # gather results
    results = workflow.gather_results()
    print(f'RESULTS: {pf(results)}')


if __name__ == '__main__':
    test_process()
